In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.utils import to_categorical 
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
  

# set seed for reproducibility
np.random.seed(0)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
from PIL import Image, ImageFilter


def imageprepare(argv):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:  # check which dimension is bigger
        # Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0 / width * height), 0))  # resize height according to ratio width
        if (nheight == 0):  # rare case but minimum is 1 pixel
            nheight = 1
            # resize and sharpen
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))  # calculate horizontal position
        newImage.paste(img, (4, wtop))  # paste resized image on white canvas
    else:
        # Height is bigger. Heigth becomes 20 pixels.
        nwidth = int(round((20.0 / height * width), 0))  # resize width according to ratio height
        if (nwidth == 0):  # rare case but minimum is 1 pixel
            nwidth = 1
            # resize and sharpen
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))  # caculate vertical pozition
        newImage.paste(img, (wleft, 4))  # paste resized image on white canvas

    # newImage.save("sample.png

    tv = list(newImage.getdata())  # get pixel values

    # normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    #print(tva)
    return tva

# precess input image: convert into numpy and reshape 
len_private_handwriting = 9 # this depends on the number of files in private-handwriting dataset
private_list = []
for i in range(1,len_private_handwriting+1):
    path = '/kaggle/input/private-handwriting/0' + str(i) + '.png'
    
    img = cv2.imread(path)
    plt.imshow(img)
    plt.show()
    
    private_list.append(imageprepare(path))

In [4]:
for im in private_list:
    plt.imshow(np.array(im).reshape(28,28))  # (rows, columns)
    # the label of the first number
    plt.show() 

## Reading the dataset

In [6]:
# read the data
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

df_train.shape, df_test.shape

In [10]:
new_df = pd.DataFrame(columns=df_test.columns)
for i in range(9):
    new_df.loc[i] = private_list[i]
new_df

In [11]:
df_test.head()

In [20]:
y = df_train['label']
y = to_categorical(y, num_classes=10) # split into 10 classes for the y value

In [21]:
y[0]

In [22]:
train = df_train.iloc[:,1:].values.reshape(-1,28,28,1)
test = df_test.iloc[:,:].values.reshape(-1,28,28,1)
new = new_df.iloc[:,:].values.reshape(-1,28,28,1)

In [23]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train[i], cmap=plt.get_cmap('gray'))

In [24]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(test[i], cmap=plt.get_cmap('gray'))

In [25]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(new[i], cmap=plt.get_cmap('gray'))

In [26]:
train = train.reshape((train.shape[0], 28*28)).astype('float64') / 255
test = test.reshape((test.shape[0], 28*28)).astype('float64') / 255
new = new.reshape((new.shape[0], 28*28)).astype('float64') / 255

In [27]:
print('Train shape:', train.shape)
print('Test shape:', test.shape)
print('New shape:', new.shape)

In [28]:
x_train, x_val , y_train, y_val = train_test_split(train, y, test_size=0.1, random_state=42) 
x_train.shape, x_val.shape, y_train.shape, y_val.shape

to plot the numbers we need to convert the first row in the dataframe to a numpy array, & then reshape it by (28,28) to look like an image.

In [29]:
# view the first number
# the reason to be 28 * 28 is that 28^2 = 784
plt.imshow(x_train[0].reshape(28, 28))  # (rows, columns)
# the label of the first number
plt.title(f"Digit: {y_train[0]}")
plt.show() 

### View an Image in More Detail

## Modeling

In [30]:
# build the model
model = Sequential()
model.add(Dense(256, input_dim=784, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

Before training a model in Keras, we need to specify an *optimizer* to perform the gradient descent, a *loss function* to be minimized, and (optionally) any *performance metrics*. The optimization algorithm we'll use for this course is called ["Adam"](https://keras.io/api/optimizers/adam/), which generally performs well regardless of what kind of problem you're trying to solve.

In [41]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
(x_train_1, y_train_1), (x_val_1, y_val_1) = mnist.load_data()
print('Train: X=%s, y=%s' % (x_train_1.shape, y_train_1.shape))
print('Test: X=%s, y=%s' % (x_val_1.shape, y_val_1.shape))

In [47]:
x_train_1 = x_train_1.reshape((x_train_1.shape[0], 28*28)).astype('float64') / 255
x_val_1 = x_val_1.reshape((x_val_1.shape[0], 28*28)).astype('float64') / 255
y_train_1 = to_categorical(y_train_1)
y_val_1 = to_categorical(y_val_1)

In [48]:
new_x_train = np.concatenate((x_train_1, x_val_1), axis=0)
new_y_train = np.concatenate((y_train_1, y_val_1), axis=0)

In [49]:
print('new_x_train shape:', new_x_train.shape)
print('new_y_train shape:', new_y_train.shape)

In [50]:
x_val, x_test , y_val, y_test = train_test_split(train, y, test_size=0.15, random_state=42)

In [51]:
optimizer = tf.keras.optimizers.Adam(0.0005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(optimizer.learning_rate)
start = time.time()
history = model.fit(new_x_train, new_y_train, epochs=40, batch_size=64, validation_data = (x_val,y_val))
end = time.time()
print("Algorithm time is {} s:".format(round(end-start)))

Generally the bigger the batch, the more stable our stochastic gradient descent updates will be. But beware of GPU memory limitations! We're going for a batch size of 128 and 25 epochs.

In [70]:
plt.rcParams["figure.figsize"] = (10,10)
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [54]:
# see the model structure
model.summary()

In [55]:
result = model.predict(x_test)

In [56]:
result

In [57]:
# evaluate the model
loss_and_metrics = model.evaluate(x_val, y_val, verbose=2)

print("Val Loss", loss_and_metrics[0])
print("Val Accuracy", loss_and_metrics[1])

In [58]:
predictions = model.predict(test)
predictions = np.argmax(predictions , axis=1)

In [59]:
submission=pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission['Label']=predictions
submission.to_csv('submission.csv', index=False)

In [60]:
test.shape

In [61]:
new.shape

In [62]:
test[0]

In [63]:
new[0]

In [65]:
new[1]

In [67]:
plt.imshow(new[1].reshape(28, 28))  # (rows, columns)
# the label of the first number
plt.title(f"Digit: ")
plt.show() 

In [68]:
plt.imshow(test[1].reshape(28, 28))  # (rows, columns)
# the label of the first number
plt.title(f"Digit: ")
plt.show() 

In [69]:
#1, 2, 3, 4, 5, 6, 7, 8
np.argmax(model.predict(new), axis=1)